# Design Assist: Transmission Line with Load

In [ ]:
include("TLines.jl")
import TLines: μ0, ϵ0
import InspectDR
using NumericIO

#Convenience aliases:
SI(x) = formatted(x, :SI, ndigits=3)

#When MIME"image/svg+xml" is disabled, Jupyter eventually requests PNG inline graphics:
#(SVG outputs do not render well in notebooks for some reason...)
InspectDR.defaults.rendersvg = false
;

# Setup: Interact, Reactive, and Signals
(Also create initial plot object)

**NOTE:** Zref=50Ω when computing reflection coefficient, Γ

In [ ]:
using Interact, Reactive

ltypelist = [("Series RLC" => :series), ("Shunt RLC" => :shunt)]
ltypemap = Dict{Symbol, String}(v=>s for (s,v) in ltypelist)
ltype = Signal(Symbol, :series) #Load type
fmin = Signal(Float64, 1e7) #Minimum plot frequency
fmax = Signal(Float64, 10e9) #Maximum plot frequency
R = Signal(Float64, 40) #Resistive load (Ω)
L = Signal(Float64, 20e-9) #Inductive load (H)
C = Signal(Float64, 0.2e-12) #Capacitive load (F)
ZC = Signal(Float64, 75) #Characteristic impedance of line (Ω)
α = Signal(Float64, 0.2) #Line attenuation (dB/m) - Nepers/m?
μ = Signal(Float64, μ0) #Permeability for line (H/m)
ϵ = Signal(Float64, ϵ0) #Permittivity for line (F/m)
ℓ = Signal(Float64, 0.1) #Line length (m)
pobj = TLines.newplot() #Create initial plot object
;

# Interact/Reactive Control: Plot limits


## TODO

In [ ]:
#txt_fmax = textbox(value(fmax), label="max freq", signal=fmax) #Will not display - notebook hangs??
#display(txt_fmin) Will hang session using this method, for some reason
txt_fmin = textbox(value(fmin), label="min freq") #This one does not cause hangups
fmin = signal(txt_fmin) #Hack to get fmax/txt_fmax working
txt_fmax = textbox(value(fmax), label="max freq")
fmax = signal(txt_fmax)
map(display, [txt_fmin, txt_fmax])
;

# Interact/Reactive Control: Inline Plots
Use sliders/text boxes to change line parameters.

## TODO
 
 - Control either RLGC or ZC, α, μ & ϵ line parameters.

In [ ]:
tgl_ltype = togglebuttons(ltypelist, value_label=ltypemap[value(ltype)], signal=ltype)
display(tgl_ltype)

#RLC sliders:
sld_R = slider(-3:.1:6, value=log10(value(R)), label="R-load: log(Ω)")
sld_L = slider(-15:.1:-3, value=log10(value(L)), label="L-load: log(H)")
sld_C = slider(-18:.1:-3, value=log10(value(C)), label="C-load: log(F)")
map(display, [sld_R, sld_L, sld_C])

#Line attenuation slider:
sld_α = slider(0:.001:1, value=value(α), label="α-DC line atten (dB/m)", signal=α)
display(sld_α)

#Line length slider:
sld_ℓ = slider(0:1e-6:1, value=value(ℓ), label="ℓ-line length (m)", signal=ℓ)
display(sld_ℓ)

#Characteristic impedance slider:
sld_ZC = slider(1:1.0:377, value=value(ZC), label="ZC (Ω)", signal=ZC)
display(sld_ZC)

calc_R = map(logR->push!(R, 10.0^logR), signal(sld_R))
calc_L = map(logL->push!(L, 10.0^logL), signal(sld_L))
calc_C = map(logC->push!(C, 10.0^logC), signal(sld_C))

display_params = map(R, L, C, ℓ) do _R, _L, _C, _ℓ
    return HTML(
        "R=" * SI(_R) * "Ω" *
        ", L=" * SI(_L) * "H" *
        ", C=" * SI(_C) * "F" *
        ", ℓ=" * SI(_ℓ) * "m"
    )
end
display(display_params)

#Widget-controlled plot:
updater_plot = map(fmin, fmax, ltype, R, L, C, ℓ, ZC, α) do _fmin, _fmax, _ltype, _R, _L, _C, _ℓ, _ZC, _α
    TLines.update(pobj, _fmin, _fmax, _ltype, _R, _L, _C, _ℓ, _ZC, _α)
    pobj
end
display(updater_plot)
;

# Interact/Reactive Control: Inspect/Gtk GUI

In [ ]:
gtkgui = display(InspectDR.GtkDisplay(), pobj) #Display plot in Gtk GUI.

#Refresh GUI when plot changes:
updater_gtkgui = map((p)->InspectDR.refresh(gtkgui), updater_plot)

#Re-display GUI controls, for convenience:
#map(display, widgetlist)
;

# Demo complete!